# Data Preparation Without Data Leakage

Data preparation is the process of transforming raw data into a form that is appropriate for modeling. A naive approach to preparing data applies the transform on the entire dataset before evaluating the performance of the model. This results in a problem referred to as data leakage, where knowledge of the hold-out test set leaks into the dataset used to train the model. This can result in an incorrect estimate of model performance when making predictions on new data. A careful application of data preparation techniques is required in order to avoid data leakage, and this varies depending on the model evaluation scheme used, such as train-test splits or $k$-fold cross-validation. In this exercise, you will discover how to avoid data leakage during data preparation when evaluating machine learning models. After completing this exercise, you will know:
- Naive application of data preparation methods to the whole dataset results in data leakage that causes incorrect estimates of model performance.
- Data preparation must be prepared on the training set only in order to avoid data leakage.
- How to implement data preparation without data leakage for train-test splits and $k$-fold cross-validation in Python.

---

## 1. Exercise Overview

This exercise is divided into three parts
1. Problem With Naive Data Preparation
2. Data Preparation With Train and Test Sets
3. Data Preparation With $k$-fold Cross-Validation

---

## 2. Problem With Naive Data Preparation

The manner in which data preparation techniques are applied to data matters. A common approach is to first apply one or more transforms to the entire dataset. Then the dataset is split into train and test sets or $k$-fold cross-validation is used to fit and evaluate a machine learning model.
1. Prepare Dataset
2. Split Data
3. Evaluate Models

Although this is a common approach, it is dangerously incorrect in most cases. The problem with applying data preparation techniques before splitting data for model evaluation is that it can lead to data leakage and, in turn, will likely result in an incorrect estimate of a model's performance on the problem. Data leakage refers to a problem where information about the holdout dataset, such as a test or validation dataset, is made available to the model in the training dataset. This leakage is often small and subtle but can have a marked effect on performance.

Data leakage means that information is revealed to the model that gives it an unrealistic advantage to make better predictions. This could happen when test data is leaked into the training set, or when data from the future is leaked to the past. Any time that a model is given information that it shouldn't have access to when it is making predictions in real time in production, there is leakage.

We get data leakage by applying data preparation techniques to the entire dataset. This is not a direct type of data leakage, where we would train the model on the test dataset. Instead, it is an indirect type of data leakage, where some knowledge about the test dataset, captured in summary statistics is available to the model during training. This can make it a harder type of
data leakage to spot, especially for beginners.

One other aspect of resampling is related to the concept of information leakage which is where the test set data are used (directly or indirectly) during the training process. This can lead to overly optimistic results that do not replicate on future
data points and can occur in subtle ways.

For example, consider the case where we want to normalize data, that is scale input variables to the range between 0 to 1. When we normalize the input variables, this requires that we fist calculate the minimum and maximum values for each variable before using these values to scale the variables. The dataset is then split into train and test datasets, but the examples in the training
dataset know something about the data in the test dataset; they have been scaled by the global minimum and maximum values, so they know more about the global distribution of the variable then they should.

We get the same type of leakage with almost all data preparation techniques; for example, standardization estimates the mean and standard deviation values from the domain in order to scale the variables. Even models that impute missing values using a model or summary statistics will draw on the full dataset to fill in values in the training dataset. The solution is straightforward. Data preparation must be fit on the training dataset only. That is, any coefficients or models prepared for the data preparation process must only use rows of data in the training dataset. Once fit, the data preparation algorithms or models can then be applied to the training dataset, and to the test dataset.
1. Split Data.
2. Fit Data Preparation on Training Dataset.
3. Apply Data Preparation to Train and Test Datasets.
4. Evaluate Models.

More generally, the entire modeling pipeline must be prepared only on the training dataset to avoid data leakage. This might include data transforms, but also other techniques such feature selection, dimensionality reduction, feature engineering and more. 

Now that we are familiar with how to apply data preparation to avoid data leakage, let's look at some worked examples.

---

## 3. Data Preparation With Train and Test Sets

In this section, we will evaluate a logistic regression model using train and test sets on a synthetic binary classfication dataset where the input variables have been normalized. First, let's define our synthetic dataset.

- Use the `make_classification()` function from `sklearn.datasets` to create the dataset `X` and `y` with 1,000 rows of data and 20 numerical input features.  
  Set the `n_informative`, `n_redundant` and `random_state` to $15$, $5$ and $7$ respectively.  
  You should expect the shape of `X` and `y` to be `(1000,20)` and `(1000,)` respectively.  
  For more information on `make_classification()`, refer to the official documentation at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html.

In [1]:
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd

In [4]:
X, y = make_classification(n_samples = 1000, n_informative = 15, n_redundant = 5, random_state = 7)
print(X.shape)
print(y.shape)

(1000, 20)
(1000,)


This creates the dataset and confirms that the input part of the dataset has 1,000 rows and 20 columns for the 20 input variables and that the output variable has 1,000 examples to match the 1,000 rows of input data, one value per row.

Next, we can evaluate our model on a scaled dataset, starting with their naive or incorrect approach.

### 3.1. Train-Test Evaluation With Naive Data Preparation

The naive approach involves first applying the data preparation method, then splitting the data before finally evaluating the model. We can normalize the input variables using the `MinMaxScaler` class, which is first defined with the default configuration scaling the data to the range $0$-$1$, then the `fit_transform()` function is called to fit the transform on the dataset and apply it to the dataset in a single step. The result is a normalized version of the input variables, where each column in the array is separately normalized (e.g. has its own minimum and maximum calculated). Don't worry too much about the specifics of this transform yet, we
will go into a more detail in the future.

```python
'''Example of configuring and applying the transform to the dataset.'''
from sklearn.preprocessing import MinMaxScaler
...
# standardize the dataset
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
```

Next, we can split our dataset into train and test sets using the `train_test_split()` function. We will use 67 percent for the training set and 33 percent for the test set.

```python
'''Example of splitting the dataset into train and test sets.'''
from sklearn.model_selection import train_test_split
...
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
```
 
We can then define our logistic regression algorithm via the `LogisticRegression` class, with default configuration, and fit it on the training dataset.

```python
'''Example of defining and fitting the model on the training dataset.'''
from sklearn.linear_model import LogisticRegression
...
# fit the model
model = LogisticRegression()
model.fit(X_train, y_train)
```

The fit model can then make a prediction using the input data from the test set, and we can compare the predictions to the expected values and calculate a classification accuracy score.

```python
'''Example of evaluating the model on the test dataset.'''
from sklearn.metrics import accuracy_score
...
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % (accuracy*100))
```

Let's tie this together.

- Using your dataset and the examples above, evaluate the model using a train-test split. (Answer: 84.848%)

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
# standardize the dataset
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# fit the model
model = LogisticRegression()
model.fit(X_train, y_train)

# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % (accuracy*100))

Accuracy: 84.848


The example above normalizes the data, splits the data into train and test sets, then fits and evaluates the model.

Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

In this case, we can see that the estimate for the model is about 84.848 percent.

<u>Given we know that there was data leakage, we know that this estimate of model accuracy is wrong.</u>

Next, let's explore how we might correctly prepare the data to avoid data leakage.

### 3.2 Train-Test Evaluation With Correct Data Preparation

The correct approach to performing data preparation with a train-test split evaluation is to fit the data preparation on the training set, then apply the transform to the train and test sets. This requires that we first split the data into train and test sets.

We can then define the `MinMaxScaler` and call the `fit()` function only on the training set, then apply the `transform()` function on the train and test sets to create a normalized version of each dataset.

```python
'''Example of fitting the transform on the train set and applying it to both train and test sets.'''
...
# define the scaler
scaler = MinMaxScaler()
# fit on the training dataset
scaler.fit(X_train)
# scale the training dataset
X_train = scaler.transform(X_train)
# scale the test dataset
X_test = scaler.transform(X_test)
```

This avoids data leakage as the calculation of the minimum and maximum value for each input variable is calculated using only the training dataset (X train) instead of the entire dataset (X). The model can then be evaluated as before.

- Tying this together, re-evaluate the model. (Answer: 85.455%)

In [7]:
# define the scaler
scaler = MinMaxScaler()
# fit on the training dataset
scaler.fit(X_train)
# scale the training dataset
X_train = scaler.transform(X_train)
# scale the test dataset
X_test = scaler.transform(X_test)

# fit the model
model = LogisticRegression()
model.fit(X_train, y_train)

# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % (accuracy*100))

Accuracy: 85.455


Running the example splits the data into train and test sets, normalizes the data correctly, then fits and evaluates the model.

Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

In this case, we can see that the estimate for the model is about 85.455 percent, which is more accurate than the estimate with data leakage in the previous section that achieved an accuracy of 84.848 percent. We expect data leakage to result in an incorrect estimate of model performance. We would expect this to be an optimistic estimate with data leakage, e.g. better performance, although in this case, we can see that data leakage resulted in slightly worse performance. This might be because of the difficulty of the prediction task.

---

## 4. Data Preparation With $k$-fold Cross-Validation

In this section, we will evaluate a logistic regression model using $k$-fold cross-validation on a synthetic binary classification dataset where the input variables have been normalized. You may recall that $k$-fold cross-validation involves splitting a dataset into $k$ non-overlapping groups of rows. The model is then trained on all but one group to form a training dataset and then evaluated on the held-out fold. This process is repeated so that each fold is given a chance to be used as the holdout test set. Finally, the average performance across all evaluations is reported. The $k$-fold cross-validation procedure generally gives a more reliable estimate of model performance than a train-test split, although it is more computationally expensive given the repeated fitting and evaluation of models. Let's first look at naive data preparation with $k$-fold cross-validation.


### 4.1. Cross-Validation Evaluation With Naive Data Preparation

Naive data preparation with cross-validation involves applying the data transforms first, then using the cross-validation procedure. 

We will use the synthetic dataset prepared in the previous section and normalize the data directly.

```python
'''Example of configuring and applying the transform to the dataset.'''
...
# standardize the dataset
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
```

The $k$-fold cross-validation procedure must first be defined. We will use repeated stratified 10-fold cross-validation, which is a best practice for classification.
- *Repeated* means that the whole cross-validation procedure is repeated multiple times, three in this case.
- *Stratified* means that each group of rows will have the relative composition of examples from each class as the whole dataset.

We will use k = 10 or 10-fold cross-validation. This can be achieved using the `RepeatedStratifiedKFold` which can be configured to three repeats and 10 folds, and then using the `cross_val_score()` function to perform the procedure, passing in the defined model,
cross-validation object, and metric to calculate, in this case, accuracy.

```python
'''Example of evaluating model performance using cross-validation.'''
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
...
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model using cross-validation
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
```

We can then report the average accuracy across all of the repeats and folds.

```python
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores)*100, std(scores)*100))
```

- Tying this all together, evaluating the regression model with cross-validation. (Answer: 85.3% (3.607))

In [8]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [11]:
'''Example of configuring and applying the transform to the dataset.'''
...
# standardize the dataset
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

'''Example of evaluating model performance using cross-validation.'''
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
...
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model using cross-validation
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)*100))

Accuracy: 85.300 (3.607)


Similarly, the example above normalizes the data first, then evaluates the model using repeated stratified $k$-fold cross-validation and reports the mean and standard deviation of the classification accuracy for the model when making predictions on data not used during training.

Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

In this case, we can see that the model achieved an estimated accuracy of about 85.3 percent, which we know is incorrect given the data leakage allowed via the data preparation procedure.

Next, let's look at how we can evaluate the model with cross-validation and avoid data leakage.

### 4.2. Cross-Validation Evaluation With Correct Data Preparation

Data preparation without data leakage when using cross-validation is slightly more challenging. It requires that the data preparation method is prepared on the training set and applied to the train and test sets within the cross-validation procedure, e.g. the groups of folds of rows. We can achieve this by defining a modeling pipeline that defines a sequence of data preparation steps to perform and ending in the model to fit and evaluate.

To provide a solid methodology, we should constrain ourselves to developing the list of preprocessing techniques, estimate them only in the presence of the training data points, and then apply the techniques to future data (including the test set).

The evaluation procedure changes from simply and incorrectly evaluating just the model to correctly evaluating the entire pipeline of data preparation and model together as a single atomic unit. This can be achieved using the `Pipeline` class. This class takes a list of steps that define the pipeline. Each step in the list is a tuple with two elements. The first element is the name of the step (a string) and the second is the configured object of the step, such as a transform or a model. The model is only supported as the final step, although we can have as many transforms as we like in the sequence.

```python
'''Example of defining a modeling pipeline.'''
from sklearn.pipeline import Pipeline
...
# define the pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('model', LogisticRegression()))
pipeline = Pipeline(steps=steps)
```

We can then pass the configured object to the `cross_val_score()` function for evaluation.

```python
'''Example of evaluating a modeling pipeline using cross-validation.'''
...
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
```

- Tying this together, re-evaluate the model.

In [12]:
'''Example of defining a modeling pipeline.'''
from sklearn.pipeline import Pipeline

In [14]:
# define the pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('model', LogisticRegression()))
pipeline = Pipeline(steps=steps)

'''Example of evaluating a modeling pipeline using cross-validation.'''
...
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)*100))

Accuracy: 85.433 (3.471)


The example above normalizes the data correctly within the cross-validation folds of the evaluation procedure to avoid data leakage.

Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

In this case, we can see that the model has an estimated accuracy of about 85.433 percent, compared to the approach with data leakage that achieved an accuracy of about 85.3 percent. As with the train-test example in the previous section, removing data leakage has resulted in a slight improvement in performance when our intuition might suggest a drop given that data leakage often results in an optimistic estimate of model performance. Nevertheless, the examples demonstrate that data leakage may impact the estimate of model performance and how to correct data leakage by correctly performing data preparation after the data is split.

---

## 5. References

- sklearn.datasets.make classification API.
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html
- sklearn.preprocessing.MinMaxScaler API.
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
- sklearn.model selection.train test split API.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
- sklearn.linear model.LogisticRegression API.
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- sklearn.model selection.RepeatedStratifiedKFold API.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html
- sklearn.model selection.cross val score API.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

Articles
- Data preparation, Wikipedia.
https://en.wikipedia.org/wiki/Data_preparation
- Data cleansing, Wikipedia.
https://en.wikipedia.org/wiki/Data_cleansing
- Data pre-processing, Wikipedia.
https://en.wikipedia.org/wiki/Data_pre-processing

---

## 6. Summary
In this exercise, you discovered how to avoid data leakage during data preparation when evaluating machine learning models. Specifically, you learned:
- Naive application of data preparation methods to the whole dataset results in data leakage that causes incorrect estimates of model performance.
- Data preparation must be prepared on the training set only in order to avoid data leakage.
- How to implement data preparation without data leakage for train-test splits and $k$-fold cross-validation in Python.